In [1]:
import pickle 
import sys
from pprint import pprint

sys.path.append('../')

from methods.SimilarityTests import SimilarityTests
from methods.LMEO import LMEO

st = SimilarityTests()

## Consistency Test.

Here we conduct the consist four experiment to determine the similarity of the rankings. We will begin by analysing London and then we will automate the process so it can easily be tested on different product and services.

### Getting the rankings.

We need to collect the following data:
- Top5 - Already got.
- Top3 - Already got.
- Top1 - Need to order.
- Pairwise - Need to order.



In [2]:
## Geting the top1 prompt
all_prompt_dict = st.get_prompt_dict()
top1_prompt = all_prompt_dict['top1'].format('London')
pairwise_prompt = all_prompt_dict['pairwise']

london_words = st.get_words_dict()['London']

print(london_words)


['Savoy', 'Langham', 'Ritz', 'Ned', 'Clarid', 'hangri', 'Dorchest', 'Shard', 'Rose', 'Royal']


In [3]:
## Ordering the top1.
london_top1 = st.order_top_1('London1000', top1_prompt, 1000)

Loading Previously Generated Ranks.


In [4]:
## Ordering the pairwise 
pairwise_words = ['The Savoy', 'The Langham', 'The Ritz', 'The Ned', 'Claridges', 'The Shangri', 'The Dorchester', 'The Shard'] # We dont have all of the hotels in here.
london_pairwise = st.order_pairwise_preferences('London1000', pairwise_prompt, pairwise_words, 50)

Loading Previously Generated Ranks.


In [5]:
## Loading the Top5 and the top3 files.

with open(r'C:\Users\chris\Documents\LMEO\LMEO\Experiments\pickles\ranks_LondonTop3_1000.pkl', 'rb') as f:
    london_top3 = pickle.load(f)

with open(r'C:\Users\chris\Documents\LMEO\LMEO\Recomendation Research\pickles\ranks_London3000.pkl', 'rb') as f:
    london_top5 = pickle.load(f)

### Processing the rankings.

In [6]:
london_top3_params = st.topk_ranks_to_params(london_top3, london_words)
london_top5_params = st.topk_ranks_to_params(london_top5, london_words)

In [7]:
london_top1_params = st.top1_ranks_to_params(london_top1, london_words, alpha=10**-10)

In [8]:
london_pairwise_params = st.pairwise_ranks_to_params(london_pairwise, pairwise_words, alpha=0)

### Comparing the results. 

In [9]:
print(london_top1_params)
print(london_top3_params)
print(london_top5_params)
print(london_pairwise_params)

[(10.381413214859949, 'Ritz'), (8.622679001064089, 'Savoy'), (8.467114893439827, 'Langham'), (7.589663994883327, 'Ned'), (7.37835490121668, 'Dorchest'), (6.685207720659676, 'OTHER'), (4.545141557207522, 'hangri'), (3.8519943766975775, 'Rose'), (-19.17385655334288, 'Clarid'), (-19.17385655334288, 'Shard'), (-19.17385655334288, 'Royal')]
[(2.8132791986731136, 'Ritz'), (0.9382143585285698, 'Langham'), (0.6417585774548971, 'Savoy'), (0.5833090506446574, 'Ned'), (0.5358631098634199, 'Shard'), (0.43687310174645144, 'Dorchest'), (0.22134039510590642, 'OTHER'), (-0.8361901850075985, 'Rose'), (-0.9198758525756158, 'Clarid'), (-1.3791314412819928, 'Royal'), (-3.0354403131518075, 'hangri')]
[(2.393704849800758, 'Ritz'), (0.5618058597780922, 'Savoy'), (0.44840088522848864, 'Langham'), (0.3396494731022665, 'Shard'), (0.18720783690111586, 'Ned'), (0.13826650128702112, 'Dorchest'), (-0.48360625500891696, 'OTHER'), (-0.4987313979795599, 'Clarid'), (-0.8572428891945337, 'Royal'), (-1.1001181145858827, 

In [10]:
# We will now use scipy kendal tau to compare the top1, top
num_to_id, id_to_num = st.get_conversions(london_words)

In [11]:
#from scipy.stats import kendalltau

top3_ordering = [id_to_num[x[1]] for x in london_top3_params]
top5_ordering = [id_to_num[x[1]] for x in london_top5_params]
top1_ordering = [id_to_num[x[1]] for x in london_top1_params]

print(top3_ordering)
print(top5_ordering)
print(top1_ordering)


[2, 1, 0, 3, 7, 6, 10, 8, 4, 9, 5]
[2, 0, 1, 7, 3, 6, 10, 4, 9, 5, 8]
[2, 0, 1, 3, 6, 10, 5, 8, 4, 7, 9]


In [12]:
import scipy.stats as stats
import itertools

top3 = ('top3', [2, 1, 0, 3, 7, 6, 10, 8, 4, 9, 5])
top5 = ('top5', [2, 0, 1, 7, 3, 6, 10, 4, 9, 5, 8])
top1 = ('top1', [2, 0, 1, 3, 6, 10, 5, 8, 4, 7, 9])

for x, y in itertools.combinations([top3, top5, top1], 2):
    print(x[0], 'against', y[0])
    print(stats.kendalltau(x[1], y[1]))

RecursionError: maximum recursion depth exceeded

In [17]:
import os

def load_all(top1_3_5_pickle_id, city_name, times):
    cf = SimilarityTests()
    lm = LMEO()
    
    # Loads the file paths checking to see if they already exists.
    
    prompt_dict = cf.get_prompt_dict()
    # Will be easier to just hard code this.
    top1_prompt = prompt_dict['top1'].format(city_name)
    top3_prompt = prompt_dict['topk'].format('three', city_name)
    top5_prompt = prompt_dict['topk'].format('five', city_name)

    # Checking for top1
    if os.path.exists(fr'.\pickles\top_1_{top1_3_5_pickle_id}.pkl'):
            print("Loading Previously Generated Top1.")
            with open(fr'.\pickles\top_1_{top1_3_5_pickle_id}', 'rb') as file:
                top1 = pickle.load(file)
    else:
        print("Generating Top1.")
        top1 = cf.order_top_1(top1_3_5_pickle_id, top1_prompt, 1000)
    
    if os.path.exists(fr'.\pickles\top_3_{top1_3_5_pickle_id}.pkl'):
            print("Loading Previously Generated Top3.")
            with open(fr'.\pickles\top_3_{top1_3_5_pickle_id}', 'rb') as file:
                top3 = pickle.load(file)
    else:
        print("Generating Top3.")
        top3 = lm.order_set(top3_prompt, top1_3_5_pickle_id, times, 3, prefix='top_3')

    if os.path.exists(fr'.\pickles\top_5_{top1_3_5_pickle_id}.pkl'):
        print("Loading Previously Generated Top5.")
        with open(fr'.\pickles\top_5_{top1_3_5_pickle_id}', 'rb') as file:
                top5 = pickle.load(file)
    else:
        print("Generating Top5.")
        top5 = lm.order_set(top5_prompt, top1_3_5_pickle_id, times, 5, prefix='top_5')

    return top1, top3, top5

def load_all(top1_3_5_pickle_id, city_name, times):
    cf = SimilarityTests()
    lm = LMEO()
    
    # Loads the file paths checking to see if they already exists.
    
    prompt_dict = cf.get_prompt_dict()
    # Will be easier to just hard code this.
    top1_prompt = prompt_dict['top1'].format(city_name)
    top3_prompt = prompt_dict['topk'].format('three', city_name)
    top5_prompt = prompt_dict['topk'].format('five', city_name)


    top1 = cf.order_top_1(top1_3_5_pickle_id, top1_prompt, 1000)
    
    top3 = lm.order_set(top3_prompt, top1_3_5_pickle_id, times, 3, prefix='top_3')

    top5 = lm.order_set(top5_prompt, top1_3_5_pickle_id, times, 5, prefix='top_5')

    return top1, top3, top5

def calculate_params(top1, top3, top5, target_words):
    print('Begining top3')
    try:
        top3_params = st.topk_ranks_to_params(top3, target_words)
    except RuntimeError:
        print("Recursion Error for top3. Using alpha")
        top3_params = st.topk_ranks_to_params(top3, target_words, alpha=10**-10)

    print('Begining top5')
    try:
        top5_params = st.topk_ranks_to_params(top5, target_words)
    except RuntimeError:
        print("Recursion Error for top5. Using alpha")
        top5_params = st.topk_ranks_to_params(top5, target_words, alpha=10**-10)
    
    print('Begining top1')
    try:
        top1_params = st.top1_ranks_to_params(top1, target_words)
    except RuntimeError:
        print("Recursion Error for top1. Using alpha")
        top1_params = st.top1_ranks_to_params(top1, target_words, alpha=10**-10)

    param_dict = {'top1': top1_params, 'top3': top3_params, 'top5': top5_params}

    num_to_id, id_to_num = st.get_conversions(target_words)

    top3_ordering = [id_to_num[x[1]] for x in top3_params]
    top5_ordering = [id_to_num[x[1]] for x in top5_params]
    top1_ordering = [id_to_num[x[1]] for x in top1_params]

    ordering_dict = {'top3': top3_ordering, 'top5': top5_ordering, 'top1': top1_ordering}

    return param_dict, ordering_dict, num_to_id
                
def do_everything(top1_3_5_pickle_id, city_name, target_words, times):
    top1, top3, top5 = load_all(top1_3_5_pickle_id, city_name, times)
    param_dict, ordering_dict, num_to_id = calculate_params(top1, top3, top5, target_words)
    return param_dict, ordering_dict, num_to_id

## Birmingham

In [18]:
birmingham_words = st.get_words_dict()['Birmingham']
birmingham_p_dict, birmingham_o_dict, birmingham_n = do_everything('Birmingham1000', 'Birmingham', birmingham_words, 1000)
print(birmingham_p_dict)
print(birmingham_o_dict)
print(birmingham_n)

Loading Previously Generated Ranks.
Loading Previously Generated Ranks.
Loading Previously Generated Ranks.


c:\Users\chris\Documents\LMEO\LMEO\Experiments\..\methods\SimilarityTests.py:20: RuntimeWarning: divide by zero encountered in log
  params = np.log(weights)
c:\Users\chris\Documents\LMEO\LMEO\Experiments\..\methods\SimilarityTests.py:21: RuntimeWarning: invalid value encountered in subtract
  return params - params.mean()


Recursion Error for top1. Using alpha
{'top1': [(9.40062702310072, 'Hyatt'), (9.05304680906799, 'almaison'), (8.665930839628217, 'dgbaston'), (8.330669990193542, 'OTHER'), (7.297654984012779, 'Vin'), (6.855822231735063, 'Grand'), (6.504424344898734, 'Marriot'), (6.45035712362875, 'Metro'), (6.393198709789129, 'Hampton'), (5.351744834971751, 'Indigo'), (3.55998536582703, 'Staying'), (-19.465865564213427, 'ACHotel'), (-19.465865564213427, 'adisson'), (-19.465865564213427, 'BirminghamCityCentre'), (-19.465865564213427, 'Cube')], 'top3': [(1.8563965978207309, 'Hyatt'), (1.7025278963445807, 'Metro'), (1.6706102108212786, 'Grand'), (0.799488442833882, 'Cube'), (0.4023150000687098, 'adisson'), (0.2975245947008942, 'almaison'), (0.15810590037076366, 'Marriot'), (0.09844730719705219, 'dgbaston'), (-0.09149291439853136, 'Staying'), (-0.36080086428459135, 'OTHER'), (-0.8281365826019932, 'ACHotel'), (-0.9046923199499842, 'Vin'), (-0.9425475712887664, 'Indigo'), (-1.8605284691647093, 'BirminghamCit

In [21]:
new_birmingham_words = ['ACHotel', 'Hyatt', 'Metro', 'Marriot', 'adisson', 'almaison', 'Vin', 'Cube', 'dgbaston', 'Grand', 'Staying', 'Indigo']
birmingham_p_dict, birmingham_o_dict, birmingham_n = do_everything('Birmingham1000', 'Birmingham', new_birmingham_words, 1000)


['ACHotel', 'Hyatt', 'Metro', 'Marriot', 'adisson', 'almaison', 'BirminghamCityCentre', 'Vin', 'Cube', 'dgbaston', 'Grand', 'Staying', 'Indigo', 'Hampton']
Loading Previously Generated Ranks.
Loading Previously Generated Ranks.
Loading Previously Generated Ranks.


c:\Users\chris\Documents\LMEO\LMEO\Experiments\..\methods\SimilarityTests.py:20: RuntimeWarning: divide by zero encountered in log
  params = np.log(weights)
c:\Users\chris\Documents\LMEO\LMEO\Experiments\..\methods\SimilarityTests.py:21: RuntimeWarning: invalid value encountered in subtract
  return params - params.mean()


Recursion Error for top1. Using alpha


In [22]:
for key, value in birmingham_o_dict.items():
    print(key, value)

top3 [1, 2, 9, 4, 7, 5, 3, 8, 10, 0, 11, 6, 12]
top5 [2, 1, 5, 6, 9, 10, 11, 8, 3, 12, 4, 7, 0]
top1 [1, 5, 8, 12, 6, 9, 3, 2, 11, 10, 0, 4, 7]


## Newcastle

In [23]:
newcastle_words = st.get_words_dict()['Newcastle']
newcastle_p_dict, newcastle_o_dict, newcastle_n = do_everything('Newcastle1000', 'Newcastle', newcastle_words, 1000)


Ordering batch 0.
Ordering batch 1.
Ordering batch 2.
Ordering batch 3.
Ordering batch 4.
Ordering batch 5.
Ordering batch 6.
Loading Previously Generated Ranks.
Loading Previously Generated Ranks.


c:\Users\chris\Documents\LMEO\LMEO\Experiments\..\methods\SimilarityTests.py:86: RuntimeWarning: invalid value encountered in divide
  params = log_transform((nums + alpha) / (denoms + alpha))


Recursion Error for top3. Using alpha
Recursion Error for top5. Using alpha


c:\Users\chris\Documents\LMEO\LMEO\Experiments\..\methods\SimilarityTests.py:20: RuntimeWarning: divide by zero encountered in log
  params = np.log(weights)
c:\Users\chris\Documents\LMEO\LMEO\Experiments\..\methods\SimilarityTests.py:21: RuntimeWarning: invalid value encountered in subtract
  return params - params.mean()


Recursion Error for top1. Using alpha


In [25]:
for key, value in newcastle_o_dict.items():
    print(key, value)
print(newcastle_n)

top3 [10, 6, 8, 2, 11, 1, 6, 7, 4, 9, 0, 5]
top5 [5, 0, 1, 2, 6, 4, 6, 7, 8, 10, 9, 11]
top1 [6, 2, 11, 0, 10, 4, 1, 9, 5, 7, 8, 6]
{0: 'Crowne', 1: 'Grey', 2: 'Malmaison', 3: 'Dene', 4: 'Vin', 5: 'County', 6: 'Dene', 7: 'Sandman', 8: 'Indigo', 9: 'Hilton', 10: 'Jury', 11: 'OTHER'}


## Bristol

In [26]:
bristol_words = st.get_words_dict()['Bristol']
bristol_p_dict, bristol_o_dict, bristol_n = do_everything('Bristol1000', 'Bristol', bristol_words, 1000)

Ordering batch 0.
Ordering batch 1.
Ordering batch 2.
Ordering batch 3.
Ordering batch 4.
Ordering batch 5.
Ordering batch 6.
Ordering batch 0.
Ordering batch 1.
Ordering batch 2.
Ordering batch 3.
Ordering batch 4.
Ordering batch 5.
Ordering batch 6.
Ordering batch 0.
Ordering batch 1.
Ordering batch 2.
Ordering batch 3.
Ordering batch 4.
Ordering batch 5.
Ordering batch 6.


c:\Users\chris\Documents\LMEO\LMEO\Experiments\..\methods\SimilarityTests.py:86: RuntimeWarning: invalid value encountered in divide
  params = log_transform((nums + alpha) / (denoms + alpha))
c:\Users\chris\Documents\LMEO\LMEO\Experiments\..\methods\SimilarityTests.py:20: RuntimeWarning: divide by zero encountered in log
  params = np.log(weights)


Recursion Error for top3. Using alpha
Recursion Error for top5. Using alpha


c:\Users\chris\Documents\LMEO\LMEO\Experiments\..\methods\SimilarityTests.py:21: RuntimeWarning: invalid value encountered in subtract
  return params - params.mean()


Recursion Error for top1. Using alpha


In [28]:
for key, value in bristol_o_dict.items():
    print(key, value)
print(bristol_n)

top3 [4, 0, 13, 7, 9, 12, 1, 8, 5, 6, 3, 2, 10, 11]
top5 [0, 1, 3, 4, 6, 7, 9, 10, 11, 12, 2, 5, 13, 8]
top1 [13, 5, 0, 4, 2, 3, 8, 1, 9, 6, 7, 10, 11, 12]
{0: 'Marriott', 1: 'Gainsborough', 2: 'Hilton', 3: 'Mercure', 4: 'arbour', 5: 'Radisson', 6: 'Avon', 7: 'DoubleTree', 8: 'Berkeley', 9: 'ibis', 10: 'Future', 11: 'ztec', 12: 'gabel', 13: 'OTHER'}


Other being first is a bit perculiar. Missing Bristol hotel not that deep.

In [29]:
with open(r'C:\Users\chris\Documents\LMEO\LMEO\Experiments\pickles\top_1_Bristol1000.pkl', 'rb') as f:
    bristol_top1 = pickle.load(f)

In [30]:
pprint(bristol_top1[:100])

[['The Bristol Hotel'],
 ['The Bristol Hotel'],
 ['The Bristol Hotel.'],
 ['Hotel du Vin & Bistro Bristol'],
 ['The Bristol Hotel'],
 ['Radisson Blu Hotel, Bristol'],
 ['Radisson Blu Hotel, Bristol'],
 ['The Bristol Hotel'],
 ['The Bristol Hotel'],
 ['Radisson Blu Hotel, Bristol'],
 ['The Bristol Hotel'],
 ['The Bristol Hotel'],
 ['The Bristol Hotel'],
 ['Radisson Blu Hotel, Bristol.'],
 ['Radisson Blu Hotel, Bristol'],
 ['The Bristol Hotel.'],
 ['The Bristol Hotel'],
 ['The Bristol Hotel'],
 ['The Bristol Hotel'],
 ['Radisson Blu Hotel, Bristol'],
 ['The Bristol Hotel'],
 ['Hotel du Vin & Bistro Bristol'],
 ['Radisson Blu Hotel, Bristol'],
 ['Radisson Blu Hotel, Bristol'],
 ['Radisson Blu Hotel, Bristol'],
 ['Radisson Blu Hotel, Bristol'],
 ['The Bristol Hotel'],
 ['The Bristol Hotel'],
 ['The Bristol Hotel'],
 ['The Bristol Hotel'],
 ['The Bristol Hotel'],
 ['The Bristol Hotel'],
 ['The Bristol Hotel'],
 ['Bristol Harbour Hotel & Spa'],
 ['The Bristol Hotel'],
 ['The Bristol Hotel'],